# Redes Neurais Recorrentes

### Descriçao

---
1. O objetivo deste notebook é construir uma Rede Neural Auto Enconders.
2. Utilizaremos o conjunto de dados direto do github, a qual iremos baixar e usar.

3. O problema consiste em prever agrupar e criar sistemas de recomendações a partir de avaliações de filmes
---

### Dicionário


Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
Date 	  										  	  |string     | Data da alteração |
Open														  |float    | preço da abertura                        |
High		     										  |float     | preço mais alto no dia	               |
Low | float | preco mais baixo no dia
Close | float | preco de fechamento
Volume | float | Volume total do dia
  

# Instalação dos pacotes

In [ ]:
!pip install pandas numpy scikit-learn keras torch torchvision torchaudio



# Documentação

1. **Pandas** -> [Link](https://pandas.pydata.org/docs/)
2. **Numpy** -> [Link](https://numpy.org/doc/)
3. **Scikit Learn** -> [Link](https://scikit-learn.org/stable/)
4. **Keras** -> [Link](https://keras.io/api/)
5. **TensorFlow** -> [Link](https://www.tensorflow.org/api_docs/python/tf/keras)
6. **PyTorch** -> [Link](https://pytorch.org/docs/stable/index.html)


# Obtendo o dataset

In [ ]:
!git clone https://github.com/batestin1/coding_the_future_dio_redes_neurais.git #clona o repositorio
!mv coding_the_future_dio_redes_neurais/dataset /content/ #move apenas a pasta dataset para fora do diretorio
!rm -rf coding_the_future_dio_redes_neurais #exclui o restante que nao nos interessa



# Instalando as bibliotecas




In [ ]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from keras.models import load_model #para salvar modelos do keras
#from tensorflow.keras.models import load_model #para salvar modelos do keras




# Lendo os Dataset

In [ ]:
#Importando os dados
movies = pd.read_csv('/content/dataset/ae/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/dataset/ae/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/dataset/ae/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [ ]:
movies.head(3)

In [ ]:
users.head()

In [ ]:
ratings.head()

# Preparando os dados de treino e os dados de teste

In [ ]:
training_set = pd.read_csv('/content/dataset/ae/train.csv')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('/content/dataset/ae/test.csv')
test_set = np.array(test_set, dtype = 'int')

In [ ]:
#Quantidade de usuários e filmes
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [ ]:
#Convertendo os dados em uma matriz com usuários nas linhas e filmes nas colunas
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Criando Tensors do Torch


In [ ]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Criando a arquitetura da AE

In [ ]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Treinando a AE

In [ ]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

# Testando a Rede

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('Loss de test:: '+str(test_loss/s))

# Salvando o modelo

In [ ]:
folder = 'ae/'

# Verifica se o diretório existe e, se não existir, cria o diretório
if not os.path.exists(folder):
    os.makedirs(folder)


# Salva o modelo no diretório especificado
torch.save(sae.state_dict(), os.path.join(folder, 'redes_auto_enconder.h5'))

# Importando os modelos

In [ ]:
#model = load_model(os.path.join(folder, 'redes_auto_enconder.h5'))

model = sae.load_state_dict(torch.load(os.path.join(folder, 'redes_auto_enconder.h5')))


In [ ]:
model